In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

current_wd = os.getcwd()
parent_parent = os.path.dirname(os.path.dirname(current_wd))
os.chdir(parent_parent)

from MyImports import *
hv.extension('bokeh')

os.chdir(current_wd)


seed = 42
pyro.set_rng_seed(seed)
torch.manual_seed(seed);

In [ ]:
X1 = torch.linspace(0.1,10,100, dtype=torch.float64)
X2 = torch.linspace(0.1,10,100, dtype=torch.float64)
XX1, XX2 = torch.meshgrid((X1, X2), indexing='xy')

## Objective functions 1, 2, 3, 4

In [ ]:
def _objective2D(x1, x2, noise):
    
    def _inner_objective(x1, x2):
        return 3*(1-x1)**2 * torch.exp(-x1**2 - (x2+1)**2) - 10*(0.2*x1 - x1**3 - x2**5) * torch.exp(-x1**2 - x2**2) - 1/3.* torch.exp(-(x1+1)**2 - x2**2)
    
    if noise != 0.:
        res = [_inner_objective(x1i, x2i) + torch.distributions.Normal(0, noise).sample() for x1i, x2i in zip(x1, x2)]
        
        return torch.cat(res, dim=0).reshape(-1)
    
    return _inner_objective(x1, x2).reshape(-1)


def scale(from_, to_):
    """
    Converts scale of input from_ to the scales of input to_.
    """
    a1 = from_ if from_.shape[-1] == 1 else from_ - from_.min()
    a2 = a1 / a1.max()
    a3 = a2 * abs(to_.max() - to_.min())
    a4 = a3 + to_.min()
    return a4


def Objective2D_ds_10(x1, x2=0., noise=.3):
    """
    Bounds [0, 10] for x1, [0, 10] for x2.
    """
    dtype = x1.dtype
    x1 = scale(torch.cat((x1, torch.tensor([[0], [10]], dtype=torch.float64)), dim=0), np.array([-2.5, 2.5]))[:-2]
    x2 = scale(torch.cat((x2, torch.tensor([[0], [10]], dtype=torch.float64)), dim=0), np.array([-2.5, 2.5]))[:-2]
    x2 = (x2*10./5).round() *5/10.
    
    return _objective2D(x1, x2, noise)

In [ ]:
hv.extension('bokeh')

obj1 = alpine2D_ds

def obj2(x1, x2, noise):
    return 0.8 * alpine2D_ds(x1, x2, noise)

obj3 = Objective2D_ds_10

def obj4(x1, x2, noise):
    return 0.9* Objective2D_ds_10(x1, x2, noise)

In [ ]:
y_obj1 = obj1(x1=XX1.reshape(-1,1), x2=XX2.reshape(-1,1), noise=0.)
data = [(x.item(), y.item(), z.item()) for x,y,z in zip(XX1.flatten(), XX2.flatten(), y_obj1)]
mean_heat1 = hv.HeatMap(data).opts(cmap='RdBu_r', width=430, height=400, colorbar=True,
                                   title='Objective function 1',
                                   xlabel='continuous variable',
                                   ylabel='discrete variable')

In [ ]:
y_obj2 = obj2(x1=XX1.reshape(-1,1), x2=XX2.reshape(-1,1), noise=0.)
data = [(x.item(), y.item(), z.item()) for x,y,z in zip(XX1.flatten(), XX2.flatten(), y_obj2)]
mean_heat2 = hv.HeatMap(data).opts(cmap='RdBu_r', width=430, height=400, colorbar=True,
                                   title='Objective function 2',
                                   xlabel='continuous variable',
                                   ylabel='discrete variable')

In [ ]:
y_obj3 = obj3(x1=XX1.reshape(-1,1), x2=XX2.reshape(-1,1), noise=0.)
data = [(x.item(), y.item(), z.item()) for x,y,z in zip(XX1.flatten(), XX2.flatten(), y_obj3)]
mean_heat3 = hv.HeatMap(data).opts(cmap='RdBu_r', width=430, height=400, colorbar=True,
                                   title='Objective function 3',
                                   xlabel='continuous variable',
                                   ylabel='discrete variable')

In [ ]:
y_obj4 = obj4(x1=XX1.reshape(-1,1), x2=XX2.reshape(-1,1), noise=0.)
data = [(x.item(), y.item(), z.item()) for x,y,z in zip(XX1.flatten(), XX2.flatten(), y_obj4)]
mean_heat4 = hv.HeatMap(data).opts(cmap='RdBu_r', width=430, height=400, colorbar=True,
                                   title='Objective function 4',
                                   xlabel='continuous variable',
                                   ylabel='discrete variable')

In [ ]:
fontsize = {'title':15, 'xlabel': 14, 'ylabel': 14, 'legend': 14}

(mean_heat1.opts(fontsize=fontsize) + mean_heat2.opts(fontsize=fontsize) + mean_heat3.opts(fontsize=fontsize) + 
 mean_heat4.opts(fontsize=fontsize)).cols(2)

In [ ]:
y_lim = [y_obj3.min()-0.5, y_obj3.max() + 0.5]

## Plot model

In [ ]:
input_dim = 3
X_test1 = torch.cat((XX1.reshape(-1,1),
                     XX2.reshape(-1,1),
                     torch.tensor([[1,0,0,0]], dtype=torch.float64).repeat(len(XX1.flatten()), 1)), dim=1)
X_test2 = torch.cat((XX1.reshape(-1,1),
                     XX2.reshape(-1,1),
                     torch.tensor([[0,1,0,0]], dtype=torch.float64).repeat(len(XX1.flatten()), 1)), dim=1)
X_test3 = torch.cat((XX1.reshape(-1,1),
                     XX2.reshape(-1,1),
                     torch.tensor([[0,0,1,0]], dtype=torch.float64).repeat(len(XX1.flatten()), 1)), dim=1)
X_test4 = torch.cat((XX1.reshape(-1,1),
                     XX2.reshape(-1,1),
                     torch.tensor([[0,0,0,1]], dtype=torch.float64).repeat(len(XX1.flatten()), 1)), dim=1)

X_test = torch.cat((X_test1, X_test2, X_test3, X_test4), dim=0)

In [ ]:
def getCateg(model, categ):
    X = model.X.T[2:].T
    indices = [(x == categ).all().item() for x in X]
    
    return model.X[indices], model.y[indices]

In [ ]:
def getObjective(model, X_test):
    x_next = model.next(X_test)
    if (x_next.T[2:].T == torch.tensor([1,0,0,0])).all().item():
        return obj1
    elif (x_next.T[2:].T == torch.tensor([0,1,0,0])).all().item():
        return obj2
    elif (x_next.T[2:].T == torch.tensor([0,0,1,0])).all().item():
        return obj3
    elif (x_next.T[2:].T == torch.tensor([0,0,0,1])).all().item():
        return obj4
    else:
        raise AttributeError('invalid category: ', x_next.T[2:].T)

In [ ]:
def getHmaps(init_model,
             num_iter,
             xlabel='continuous variable',
             ylabel='numerical discrete variable',
             fontsize = {'title': 15, 'xlabel': 14, 'ylabel': 14, 'legend': 14},
             width = 400,
             height = 350,
             init_samples=None):
    
    hmaps = OrderedDict()
    former_objective = obj1
    
    # outer loop: defines current categorical context
    for x_test, obj, idx in zip([X_test1, X_test2, X_test3, X_test4],
                                [obj1, obj2, obj3, obj4],
                                [0,1,2,3]):
        
        txt = 'Categ {}, '.format(idx+1)
        # middle loop: num_iter times runs per categorical context
        
        for j in range(num_iter):
            
            length_scale = init_model.kernel.lengthscale_unconstrained.exp()
            output_scale = init_model.kernel.variance_unconstrained.exp().sqrt()
            
            print(txt + 'Iteration {}: \n noise: {:.2f}, l: {:.2f}, lambda: {:.2f}'.format(j,
                                                                                            init_model.noise,
                                                                                            length_scale,
                                                                                            output_scale))
            if noise == 0.:
                lengthscale.append(length_scale)
                outputscale.append(output_scale)
                noise_.append(init_model.noise)
            else:
                lengthscale_noisy.append(length_scale)
                outputscale_noisy.append(output_scale)
                noise_noisy.append(init_model.noise)
            if j!= 0 or former_objective != obj:
                y_next = former_objective(x1=x_next.T[0:1].T, x2=x_next.T[1:2].T, noise=noise)
                init_model.add_Observation(x_next, y_next)
                init_model.update()
                
            mean_hmaps = []
            
            # inner loop: plot all categories
            for x_test_inner, obj_inner, title in zip([X_test1, X_test2, X_test3, X_test4],
                                                      [obj1, obj2, obj3, obj4],
                                                      [1,2,3,4]):
                data = [(x.item(), y.item(), z.item())
                        for x,y,z in zip(x_test.T[0].T,
                                         x_test.T[1].T,
                                         init_model(x_test_inner)[0].data)]
                
                X, _ = getCateg(init_model, x_test_inner[0].T[2:].T)
                
                train_points = hv.Points((X.T[0].T, X.T[1]), label='Iterate Points').opts(color='black', size=5)
                if init_samples is not None:
                    train_points = train_points * hv.Points((init_samples[(title-1)*20: title*20+20].T[0].T,
                                                             init_samples[(title-1)*20: title*20+20].T[1].T),
                                                            label='Initial Points').opts(color='orange', size=5)
                new_plot = hv.HeatMap(data).opts(cmap='RdBu_r', width=width, height=height, colorbar=True,
                                                 title='Mean prediction {}:\n {:.4f}'.format(title,
                                                                                              init_model.kernel.module_2.weight.T[title-1].T.item()),
                                                 xlabel=xlabel,
                                                 ylabel=ylabel)
                
                data = [(x.item(), y.item(), z.item()) for x,y,z in zip(x_test.T[0].T,
                                                                        x_test.T[1].T,
                                                                        init_model(x_test_inner)[1].data)]
                
                unc_plot = hv.HeatMap(data).opts(cmap='RdBu_r', width=width, height=height, colorbar=True,
                                                 title='Uncertainty {}'.format(title),
                                                 xlabel=xlabel,
                                                 ylabel=ylabel)
                
                if (x_test == x_test_inner).all():
                    x_next = init_model.next_sample(x_test).data
                    next_plot = hv.Points(x_next, label='Next sample').opts(marker='diamond', color='#999999', size=23)
                    
                    mean_hmaps.append(
                        ((new_plot*next_plot*train_points).opts(show_legend=False, fontsize=fontsize)
                         + (unc_plot*next_plot*train_points).opts(show_legend=False, fontsize=fontsize)))
                    
                    acq_values = init_model.get_acq_values(x_test)
                    data = [(x.item(), y.item(), z.item()) for x,y,z in zip(x_test.T[0].T, x_test.T[1].T, acq_values)]
                    acq_hmap = hv.HeatMap(data).opts(cmap='plasma', width=width, height=height+80, colorbar=True,
                                                     title='Acquisition Function (categ {})'.format(title),
                                                     xlabel=xlabel,
                                                     ylabel=ylabel, fontsize=fontsize) * next_plot * train_points
                    acq_hmap = acq_hmap.opts(show_legend=True, legend_position='bottom')
                else:
                    mean_hmaps.append((new_plot*train_points).opts(show_legend=False, fontsize=fontsize)
                                      + (unc_plot*train_points).opts(show_legend=False, fontsize=fontsize))
                    
            hmaps[len(hmaps)] = (  mean_hmaps[0][0].opts(fontsize=fontsize) + mean_hmaps[0][1]
                                 + mean_hmaps[1][0] + mean_hmaps[1][1]
                                 + mean_hmaps[2][0] + mean_hmaps[2][1]
                                 + mean_hmaps[3][0] + mean_hmaps[3][1]
                                 + acq_hmap).opts(shared_axes=False).cols(2)
            
            former_objective = obj
            
            
    return hmaps

## extended BO

In [ ]:
# 20 initial samples per category, 80 in total
num_init = 20
noise = 0.

X_init1 = torch.cat((torch.rand(num_init, 1, dtype=torch.float64)*10,
                     torch.rand(num_init, 1, dtype=torch.float64)*10,
                     torch.tensor([1,0,0,0]).repeat(num_init,1)), dim=1)
y_init1_nf = obj1(x1=X_init1.T[0:1].T, x2=X_init1.T[1].T.reshape(-1,1), noise=noise)

X_init2 = torch.cat((torch.rand(num_init, 1, dtype=torch.float64)*10,
                     torch.rand(num_init, 1, dtype=torch.float64)*10,
                     torch.tensor([0,1,0,0]).repeat(num_init,1)), dim=1)
y_init2_nf = obj2(x1=X_init2.T[0:1].T, x2=X_init2.T[1].T.reshape(-1,1), noise=noise)

X_init3 = torch.cat((torch.rand(num_init, 1, dtype=torch.float64)*10,
                     torch.rand(num_init, 1, dtype=torch.float64)*10,
                     torch.tensor([0,0,1,0]).repeat(num_init,1)), dim=1)
y_init3_nf = obj3(x1=X_init3.T[0:1].T, x2=X_init3.T[1].T.reshape(-1,1), noise=noise)

X_init4 = torch.cat((torch.rand(num_init, 1, dtype=torch.float64)*10,
                     torch.rand(num_init, 1, dtype=torch.float64)*10,
                     torch.tensor([0,0,0,1]).repeat(num_init,1)), dim=1)
y_init4_nf = obj4(x1=X_init4.T[0:1].T, x2=X_init4.T[1].T.reshape(-1,1), noise=noise)

X_init = torch.cat((X_init1, X_init2, X_init3, X_init4), dim=0)
y_init_nf = torch.cat((y_init1_nf, y_init2_nf, y_init3_nf, y_init4_nf), dim=0)

In [ ]:
input_dim = 3
kappa = 10.

kernel = TransformationKernel(input_dim = input_dim,
                              trans_mappings=OrderedDict({(0,): lambda x:x,
                                                          (1,): lambda x:x.round(),
                                                          (2,3,4,5): torch.nn.Linear(4,1,bias=False, dtype=torch.float64)}))

bo_model_nf = PyroBO(X_init, y_init_nf, kernel=kernel, acq_fun=ConfidenceBound(input_dim=input_dim,
                                                                               kappa=kappa,
                                                                               maximize=True))
bo_model_nf.update()

In [ ]:
num_iter=4 # number of iterations per category
noise = 0.

lengthscale = []
outputscale = []
noise_ = []

hmap_nf = hv.HoloMap(getHmaps(bo_model_nf, num_iter=num_iter, init_samples=X_init), kdims='iteration')
hv.output(hmap_nf.collate(), widget_location='top')

In [ ]:
hv.save(hmap_nf.collate(), 'Combine_All.auto')

In [ ]:
hv.extension('plotly')

lengthscale_nf_plot = hv.Curve((torch.tensor(lengthscale))).opts(height=200)
outputscale_nf_plot = hv.Curve((torch.tensor(outputscale))).opts(height=200)
noise_nf_plot = hv.Curve((torch.tensor(noise_))).opts(height=200)

(lengthscale_nf_plot + outputscale_nf_plot + noise_nf_plot).opts(shared_axes=False).cols(1)

## extended BO - noisy data

In [ ]:
noise = 0.5

y_init1_noisy = obj1(x1=X_init1.T[0:1].T, x2=X_init1.T[1].T.reshape(-1,1), noise=noise)

y_init2_noisy = obj2(x1=X_init2.T[0:1].T, x2=X_init2.T[1].T.reshape(-1,1), noise=noise)

y_init3_noisy = obj3(x1=X_init3.T[0:1].T, x2=X_init3.T[1].T.reshape(-1,1), noise=noise)

y_init4_noisy = obj4(x1=X_init4.T[0:1].T, x2=X_init4.T[1].T.reshape(-1,1), noise=noise)

y_init_noisy = torch.cat((y_init1_noisy, y_init2_noisy, y_init3_noisy, y_init4_noisy), dim=0)

In [ ]:
kernel = TransformationKernel(input_dim = input_dim,
                              trans_mappings=OrderedDict({(0,): lambda x:x,
                                                          (1,): lambda x:x.round(),
                                                          (2,3,4,5): torch.nn.Linear(4,1,bias=False,
                                                                                     dtype=torch.float64)}))

bo_model_noisy = PyroBO(X_init, y_init_noisy, kernel=kernel, acq_fun=ConfidenceBound(input_dim=input_dim,
                                                                                     kappa=kappa,
                                                                                     maximize=True))
bo_model_noisy.update()

In [ ]:
hv.extension('bokeh')

lengthscale_noisy = []
outputscale_noisy = []
noise_noisy = []

hmap_noisy = hv.HoloMap(getHmaps(bo_model_noisy, num_iter=num_iter, init_samples=(X_init)), kdims='iteration')
hv.output(hmap_noisy.collate(), widget_location='top')

In [ ]:
hv.save(hmap_noisy.collate(), 'Combine_All_noisy.auto')

In [ ]:
hv.extension('plotly')

lengthscale_noisy_plot = hv.Curve((torch.tensor(lengthscale_noisy))).opts(height=200)
outputscale_noisy_plot = hv.Curve((torch.tensor(outputscale_noisy))).opts(height=200)
noise_noisy_plot = hv.Curve((torch.tensor(noise_noisy))).opts(height=200)

(lengthscale_noisy_plot + outputscale_noisy_plot + noise_noisy_plot).opts(shared_axes=False).cols(1)